# DSC Unit 5.4 API Mini Project

BEFORE RUNNING SCRPIT FILL IN API KEY IN THE CELL BELOW

In [1]:
API_KEY='---' #Make sure to fill API_KEY

# This particular url string will select all the data from the AFX_X stock for every avaliable day in 2017 in a json
# in ascending order
url='https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31&order=asc&api_key={}'.format(API_KEY)


### Task 1

Importing request and collection libraries

In [2]:
import requests
import collections

### Task 2

Getting the 2017 data and converting it to a dict

In [3]:
r=requests.get(url ) #getting data from url 

json_data=r.json() # converting json to dict im calling json_data


For convience, the column names and data lists from json_data are respectively saved in their own variables. The column names are then zipped with each list in data (where each list in data represents the values of the day for their corresponding label). This is so that accessing each value in a given row of data is understandable (e.g data[0]['Open'] is the opening price of the first day represented in data)


In [4]:
col_names=json_data['dataset']['column_names'] # this list contains labels like Date,Open,High, Low etc.


# Below is a list of lists, data. Each list contains the value corresponding to the label in col_names in the same position
#e.g the 2nd element in any of the listsin data is the Open price, because Open is the 2nd element of col_names

data= json_data['dataset']['data'] 

# The below line changes each list in data into a dict where each value can be accessed by the label its associated with
# e.g to get the 
data= [dict(zip(col_names,list))for list in data] 

### Task 3 

A list of all the valid opening prices in 2017 (opening_prices) is created, the sorted in ascending order and the first and last elements are returned. These returned elements are the minimum opening price and maximum opening price respectively.

In [5]:
# The list comprehension below returns the opening price if it is not a Nonetype (meaning it's a float)

opening_prices=[list['Open'] for list in data if list['Open'] is not None ] 

sorted(opening_prices) #sorting opening prices in ascending order

opening_price_low=opening_prices[0] #first element is lowest opening price

opening_price_high=opening_prices[-1]#last element is lowest opening price


print('lowest opening price is  : $',end='')
print(opening_price_low)

print('highest opening price is : $',end='')
print(opening_price_high)



lowest opening price is  : $34.99
highest opening price is : $51.76


### Task 4

The daily fluctuations are calculated using a list comprehension of data that returns the absolute difference between the daily high and low for each row of data. The daily fluctuations list is then sorted in ascending order and the last element is returned for the highest fluctuation recorded in the year. 

In [6]:
# Below is a list comprehension that takes the absolute difference between the high and low of each row in data and 
# rounds the result to 2 decimal places (this is necessary because the subtraction of 2 floating point number returns
# a floating point number with many non significant decimal places)
day_fluctuation= [round(abs(day['High']-day['Low']),2) for day in data] 

fluctuation_high= sorted(day_fluctuation)[-1] #sorting daily fluctuations

print('The largest daily fluctuation in 2017 is : $',end='')
print(fluctuation_high)

The largest daily fluctuation in 2017 is : $2.81


### Task 5

1. To calculate the change in Closing prices between days, a list comprehension over range(0,len(data)-1) was created so that the closing price value for every row (day) and the next could be accessed and the absolute difference taken. 

2. Using another list comprehension, a list of strings were created that showed which days the previous list comprehension calculated the closing price over. This was to test a hypothesis: because some days in the data were missing and I was curious if the largest change in price corresponded to a period of time where a lot of data was missing. 

3. Zipping the list of strings representing the dates the closing price changes were calculated over with the list of changes in closing price and passing the resulting zip object to the list function created a list of tuples where the first elements were said dates of calculation and the second element was the corresponding change in closing price.

4. This list of tuples is sorted using a lambda function as the key for the sorted function to sort on the second element in the tuples (change in closing price) in ascending order. The last element is returned for the largest change in closing price between days.

5. The output string is formated to reveal the largest change and the dates for which is was calculated. My hypothesis was wrong because the dates of the largest change were consecutive.

In [7]:

inter_days_closing_change=[round(abs(data[day_index]['Close']-data[day_index+1]['Close']),2)for day_index in range(len(data)-1)]
# 1. calculating absolute difference between every consecutive rows' closing prices and saving each calculation in list


days_change_calculated_over= ['{}_and_{}'.format(data[day_index]['Date'],data[day_index+1]['Date']) for day_index in range(len(data)-1)]
# 2. creating a list of strings that show which days the above list comprehension calculates the change between

list_closing_change_with_day=list(zip(days_change_calculated_over,inter_days_closing_change))
# 3. zipping together the dates and changes and passing to list to create list of tuples

largest_change=sorted(list_closing_change_with_day,key= lambda x: x[1])[-1]
# 4. sorting over second elements of tuples in list of tuples (that is sorting over closing price change) in ascending order 
#    then returning the last element for largest change

# 5. formating output string to show largest change with dates calculated over
output_string='{} between days: {}'.format(largest_change[1],largest_change[0])

print('The largest change between closing prices of recorded days in 2017 is $: ',end='')
print(output_string)

The largest change between closing prices of recorded days in 2017 is $: 2.56 between days: 2017-08-08_and_2017-08-09


### Task 6 

Average volume is calculated by first calculating the sum of the Traded Volume values in every row of data, then dividing that sum by the length of data (number of rows).

In [8]:
Traded_volume_sum= 0

for day in data:
    Traded_volume_sum+=day['Traded Volume'] #adding the Traded Volume to running sum Traded_volume_sum
    
Traded_volume_avg=round(Traded_volume_sum/len(data)) #Dividing by number of rows in data and rounding to get average

print('Average Trade volume is:',end=' ')
print(Traded_volume_avg)

Average Trade volume is: 89124


### Task 7

The median is calculated by first making a sorted list in ascending order of Traded volumes using a list comprehension. The to get the index of the median of the list, it is determined whether the list is of odd or even length (using %2 on the length of Traded_volume), and the index before the median's index is taken (called middle, see Middle Calculation Note). With these 2 results the median can be calculated (See Median Calculation Note)

Middle Calculation Note: int(len(Traded_volume))/2 takes the length of Traded_volume, divides it by two and by converting the result to an int the index before the median is recieved.

    Odd example: if length is 5 the median index will be 3, middle is 2
                5/2=2.5
                int(2.5)=2
                2+1=3
    
    Even example: if length is 4, the median will be calculated using the 2nd and 3rd index, middle is 2 
                4/2=2
                int(2)=2
                2+1=3
                so index 2 and 3 can be accessed.
                
Median Calculation Note: 
(is_odd*Traded_volume[middle+1]+(1-is_odd)*((Traded_volume[middle]+Traded_volume[middle+1])/2)) calculates the median
     
            1. This formula is in form is in (is_odd*if_odd_expression+ (1-is_odd)*if_even_expression)
            
            2. is_odd is either 0 or 1,so
            
                 if length is odd: 
                 
                         (is_odd*if_odd_expression+ (1-is_odd)*if_even_expression) 
                         
                                    =1*if_odd_expression+0*if_even_expression 
                                    
                                               =if_odd_expression
                                               
                  if length is even: 
                 
                         (is_odd*if_odd_expression+ (1-is_odd)*if_even_expression) 
                         
                                    =0*if_odd_expression+1*if_even_expression 
                                    
                                               =if_even_expression
                                               
                                               
             3. if_odd_expression= Traded_Volume[middle+1], middle is the index before median's index so
                                            
                                       Traded_Volume[middle+1]=Traded[median_index]   
             
             4. if_even_expression= (Traded_volume[middle]+Traded_volume[middle+1])/2: The average
                of the two middle indices, because middle is the index before the median's index (or would be
                median index) and the middle is the one after 


        

In [21]:
Traded_volume= sorted([day['Traded Volume'] for day in data]) #sorted list of Trading Volumes

# index before median if length is odd, or the middle left index if even see Middle Calculation Note in cell above
middle=int(len(Traded_volume)/2)
                                
is_odd=len(Traded_volume)%2 # determines if length is even or odd 


# If length is odd, median is returned, if length even average of middle values returned. See Median Calculation Note
median= (is_odd*Traded_volume[middle+1]+(1-is_odd)*((Traded_volume[middle]+Traded_volume[middle+1])/2))


print('Median Trade volume is:',end=' ')
print(median)

Median Trade volume is: 76600.0
